# 一、重建索引（Reindex）

# 二、使用drop函数从一条轴上删除元素

## 1.删除Series中的元素

In [2]:
import numpy as np
import pandas as pd
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [16]:
new_obj = obj.drop('c')  #删掉索引为c的元素
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [13]:
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [17]:
obj.drop(['d', 'c']) #删除d、c两个元素

a    0.0
b    1.0
e    4.0
dtype: float64

## 2.指定轴删除DataFrame的行或列

In [6]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),index=['Ohio', 'Colorado', 'Utah', 'New York'],columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [7]:
data.drop(['Colorado', 'Ohio']) #沿0轴（行）删除，即根据行的索引删除行

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [8]:
data.drop('two', axis=1)  #沿1轴（列）删除，跟根据列的标签删除指定列

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [9]:
data.drop(['two', 'four'], axis='columns')  #axis参数可以指定为columns，与指定为1作用相同。

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


In [18]:
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [19]:
obj.drop('c', inplace=True)  #就地修改原对象，使用inplace参数需小心，因为被删掉的元素都被销毁了。
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

# 三、索引、选择、筛选

##  对Series索引操作

In [21]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [22]:
obj['b'] #按照索引标签定位对象

1.0

In [23]:
obj[1] #也可以按照位置进行索引

1.0

In [24]:
obj[2:4] #切片

c    2.0
d    3.0
dtype: float64

In [25]:
obj[['b', 'a', 'd']] #多个索引标签

b    1.0
a    0.0
d    3.0
dtype: float64

In [26]:
obj[[1, 3]]  #多个索引位置

b    1.0
d    3.0
dtype: float64

In [27]:
obj[obj < 2]  #使用条件表达式

a    0.0
b    1.0
dtype: float64

**Series使用索引标签切片与普通Numpy切片不同之处是Series切片时最后的元素是包含在内的。**

In [28]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

**使用切片表达式对多个元素进行赋值**

In [30]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

## 对DataFrame的切片、索引操作生成一列或多列元素。

In [32]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),index=['Ohio', 'Colorado', 'Utah', 'New York'],columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


**使用列标签进行索引，得到对应的列**

In [33]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [34]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


**行选择**

In [35]:
data[:2]  #行选择的简便形式

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [44]:
data[data['three'] > 5]  #逻辑表达式，选择行

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


**使用布尔DataFrame进行选择，用法与Numpy的二维数组是类似的。**

In [46]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [47]:
data[data < 5] = 0  #小于5的元素赋值为0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


##  使用loc或iloc进行行选择
loc是用索引标签进行行选择，iloc使用整数索引进行行选择。

In [48]:
data.loc['Colorado', ['two', 'three']] #选择Colorado行的two和three这两列的对应元素

two      5
three    6
Name: Colorado, dtype: int32

In [49]:
data.iloc[2, [3, 0, 1]]   #选择第3行的four、one、two这三列的对应元素

four    11
one      8
two      9
Name: Utah, dtype: int32

In [50]:
data.iloc[2] #第三行

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [51]:
data.iloc[[1, 2], [3, 0, 1]] #第2、3行的第4、1、2列的对应元素

,four,one,two
Colorado,7,0,5
Utah,11,8,9


**上面的对一个或多个标签进行的索引的方法也可应用于切片**

In [52]:
data.loc[:'Utah', 'two'] #Utah行之前的行的two列元素

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int32

In [54]:
data.iloc[:, :3][data.three > 5] #先选择所有的行，前3列，再使用逻辑表达式筛选

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


### DataFrame的索引类型
|类型| 说明|
|---|---|
|df[val] | 选择单列或多列，特殊用法：布尔值数组，切片，布尔值DataFrame|
|df.loc[val] |根据标签选择行|
|df.loc[:, val] | 根据标签选择列|
|df.loc[val1, val2] | 同时选择行、列|
|df.iloc[where]| 根据整数位置选择行|
|df.iloc[:, where] | 根据整数位置选择列| 
|df.iloc[where_i, where_j] | 根据整数位置同时选择行、列|
|df.at[label_i, label_j]| 根据行、列标签选择一个标量值|
|df.iat[i, j] | 根据整数位置选择标量值|
|reindex | 根据行、列标签选择|
|get_value, set_value| 根据行列标签读取值或赋值|

# 四、整数索引产生的问题

In [63]:
ser = pd.Series(np.arange(3.))  #不适用索引标签，索引缺省为整数序列
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [66]:
ser[2]  #因为是整数标签，使用的是pandas Series中的索引对象进行索引。2是索引对象[0,1,2]中的元素，所以能正常读取。

2.0

**没有标签的情况下，使用整数位置索引，用-1取最后一个数，产生错误。因为Ser中的索引为整数0,1,2，没有-1。在python的内建索引机制中，-1是取最后一个数，而pandas是按照其索引对象来定位的，所以python内建的索引机制与pandas的索引机制在这里产生了混淆。**

In [60]:
ser[-1]   # Series索引对象[0,1,2]中不包含-1,因此出错。

KeyError: -1

In [58]:
ser2 = pd.Series(np.arange(3.), index=['a', 'b', 'c'])

**使用标签的Series，因为Series的索引对象不是整数，因此使用整数索引时不会产生混淆，只可能用python内建索引机制进行选择，选择是最后一个元素。所以，为避免索引对象的混淆，使用索引标签是推荐行为。**

In [59]:
ser2[-1]   #与ser不同，这里是python内建索引机制，因此-1是读取最后一个数

2.0

In [61]:
ser[:1]  #同样，切片操作也是面向标签的。所以，因为Ser是整数索引，没有索引标签，所以这里选择的Series的整数标签行，而不是python的内建索引机制。

0    0.0
dtype: float64

In [62]:
ser.loc[:1]  #如果用pandas的loc函数，则选择对象的意义很明确，loc函数的结果包含最后的数的

0    0.0
1    1.0
dtype: float64

 # 五、数学运算与数据对齐
 pandas的一个重要特性是两个不同索引的对象进行数学运算会如何表现。如，这两个对象的索引不同，讲这两个对象相加，如果索引对不相同，那么结果对象的索引是参加运算的两个对象的索引的并集。如果你对数据库熟悉的话，则可以理解成索引标签的outer连接。

In [67]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [69]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1],index=['a', 'c', 'e', 'f', 'g'])
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [70]:
s1 + s2  #索引标签d只在s1中，f、g只在s2中，结果是两个合集

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

###  对DataFrame来说，行和列两个方向都要对齐

In [71]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),index=['Ohio', 'Texas', 'Colorado'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [72]:
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [73]:
df1 + df2   #两者之和，只在索引标签和列标签并集的位置数值相加，其他位置都是NaN

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


所以，如果两个DataFrame没有共同的索引标签或行标签，加（减）运算的结果将全为NaN。

In [76]:

df1 = pd.DataFrame({'A': [1, 2]})
df1

,A
0,1
1,2


In [77]:
df2 = pd.DataFrame({'B': [3, 4]})
df2

,B
0,3
1,4


In [79]:
df1 - df2  #两者没有共同的列，所以所有的数值都是NaN

,A,B
0,NaN,NaN
1,NaN,NaN


## 以填充数值进行算数运算
对不同索引的对象进行算数运算，如果一个对象轴中的数据在另一个对象中找不到对应的标签，可以指定填充值进行填充。

In [4]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),columns=list('abcd'))
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [8]:
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),columns=list('abcde'))
df2.loc[1, 'b'] = np.nan   #df2的1行b列有一个数值缺失。
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [9]:
df1 + df2   #缺失值的计算仍是缺失值

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [10]:
df1.add(df2, fill_value=0) #对df1和df2中的缺失值指定为0，然后再计算。

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


### 对于Series和DataFrame的Reindex函数，也可以指定缺失填充值

In [11]:
df1.reindex(columns=df2.columns, fill_value=0) #对df1使用df2的列名重新创建索引，缺失的列填充为0.

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


### 灵活的算数运算函数
|函数| 描述|
|---|---|
|add, radd | 相加，radd是把对象翻为右参数，下同|
|sub, rsub | 相减|
|div, rdiv | 相除|
|floordiv, rfloordiv | 地板除(//)  |
|mul, rmul | 乘法| 
|pow, rpow | 指数|

In [12]:
1 / df1

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250000,0.200000,0.166667,0.142857
2,0.125000,0.111111,0.100000,0.090909


In [13]:
df1.rdiv(1)  #df1.rdiv(1)与1/df1等价。

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250000,0.200000,0.166667,0.142857
2,0.125000,0.111111,0.100000,0.090909


## DateFrame和Series之间的运算
与numpy支持不同维度的数组之间的运算类似，pandas也支持DataFrame和Series之间的运算。

In [14]:
arr = np.arange(12.).reshape((3, 4))
arr

array([[  0.,   1.,   2.,   3.],
       [  4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.]])

In [15]:
arr[0]

array([ 0.,  1.,  2.,  3.])

In [16]:
arr - arr[0]  #DataFrame与其第一行的差

array([[ 0.,  0.,  0.,  0.],
       [ 4.,  4.,  4.,  4.],
       [ 8.,  8.,  8.,  8.]])

上例中，Numpy数组的每一行都会减去第一行，而不是只是第一行参加运算。这就是**广播（broadcasting）**。在pandas中，也有类似广播模式。

In [17]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),columns=list('bde'),index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [19]:
series = frame.iloc[0]
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

In [20]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


缺省状况下，DataFrame与Series的差是以Series的索引去匹配DataFrame的列，逐行广播运算。
如果一个索引的值在DataFrame列中或Series索引之一中不存在，那么结果对象会合并DataFrame列名及Series索引名以重建为新的索引。

In [22]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


如果想要改变上述缺省行为，想要DataFrame按列广播，在行上进行匹配，那么可以用算数函数，并指定轴参数加以改变。

In [23]:
series3 = frame['d']
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [24]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [25]:
frame.sub(series3, axis='index')  #指定按列进行运算并广播运算。

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


# 六、函数应用及映射